In [1]:
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
df

,Кол-во отправленных сообщений,Кол-во полученных сообщений,Кол-во Адресатов,Кол-во поздно прочитанных сообщений,Среднее кол-во дней между получением и прочтением,Кол-во отвеченных сообщений,Кол-во символов текста в сообщениях,Соотношение полученных и отправленных,Вердикт
0,245,71,18,40,19,11,20,3.450704,0
1,1704,381,28,84,10,282,14,4.472441,0
2,93,462,134,90,17,397,24,0.201299,1
3,57,1353,15,36,12,154,39,0.042129,0
4,73,1111,38,50,17,36,66,0.065707,0
...,...,...,...,...,...,...,...,...,...
995,1169,368,27,2,13,204,1,3.176630,1
996,391,504,5,2,6,319,3,0.775794,0
997,726,89,139,0,75,33,12,8.157303,0
998,1321,619,1,78,18,156,47,2.134087,1


In [4]:
df = pd.read_csv("GENERATED_DATASET.csv")

In [40]:
df = df.drop(columns='Unnamed: 0')

In [41]:
from sklearn.model_selection import train_test_split

train, val, train_target, val_target = train_test_split(df.drop(columns=['Вердикт']), df['Вердикт'], test_size=0.15)

In [42]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

def objective(trial):
    params = {
        "iterations" : 1000,
        "learning_rate" : trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth" : trial.suggest_int("depth", 4, 10),
        "subsample" : trial.suggest_float("subsample", 0.05, 1.0),
        "l2_leaf_reg" : trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5),
        "min_child_samples" : trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "eval_metric" : "F1"
    }

    model = CatBoostClassifier(**params)

    model.fit(train, train_target, eval_set=[(val, val_target)], verbose=0)

    preds = model.predict(val)
    score = f1_score(val_target, preds)

    return score

In [47]:
import optuna 

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2023-12-09 17:12:55,184] A new study created in memory with name: no-name-cf02ac63-56b8-4108-b9f1-1e0c86429a93
[I 2023-12-09 17:13:01,291] Trial 0 finished with value: 0.3488372093023256 and parameters: {'learning_rate': 0.03421556511017629, 'depth': 10, 'subsample': 0.8256261220860218, 'l2_leaf_reg': 3.0, 'min_child_samples': 16, 'colsample_bylevel': 0.2138072768712157}. Best is trial 0 with value: 0.3488372093023256.
[I 2023-12-09 17:13:03,987] Trial 1 finished with value: 0.3333333333333333 and parameters: {'learning_rate': 0.07298868784139152, 'depth': 7, 'subsample': 0.9896755691385893, 'l2_leaf_reg': 2.5, 'min_child_samples': 4, 'colsample_bylevel': 0.18629501560648276}. Best is trial 0 with value: 0.3488372093023256.
[I 2023-12-09 17:13:06,624] Trial 2 finished with value: 0.03125 and parameters: {'learning_rate': 0.0024697860332279177, 'depth': 5, 'subsample': 0.40145283471468957, 'l2_leaf_reg': 2.0, 'min_child_samples': 1, 'colsample_bylevel': 0.3589031600577912}. Best is t

In [48]:
cb = CatBoostClassifier(**study.best_params)

In [53]:
cb.fit(train, train_target)

0:	learn: 0.6847921	total: 13.2ms	remaining: 13.2s
1:	learn: 0.6716791	total: 21.5ms	remaining: 10.7s
2:	learn: 0.6598227	total: 29.3ms	remaining: 9.75s
3:	learn: 0.6552196	total: 31.5ms	remaining: 7.83s
4:	learn: 0.6465511	total: 39.1ms	remaining: 7.77s
5:	learn: 0.6344598	total: 47.4ms	remaining: 7.86s
6:	learn: 0.6320185	total: 49.1ms	remaining: 6.97s
7:	learn: 0.6296744	total: 51.6ms	remaining: 6.4s
8:	learn: 0.6223407	total: 60.1ms	remaining: 6.62s
9:	learn: 0.6137708	total: 66.9ms	remaining: 6.62s
10:	learn: 0.6057245	total: 73.8ms	remaining: 6.63s
11:	learn: 0.5977645	total: 82.8ms	remaining: 6.82s
12:	learn: 0.5960780	total: 86.3ms	remaining: 6.55s
13:	learn: 0.5890799	total: 94.9ms	remaining: 6.68s
14:	learn: 0.5844076	total: 133ms	remaining: 8.76s
15:	learn: 0.5774533	total: 145ms	remaining: 8.89s
16:	learn: 0.5769311	total: 150ms	remaining: 8.68s
17:	learn: 0.5720341	total: 174ms	remaining: 9.51s
18:	learn: 0.5717627	total: 176ms	remaining: 9.1s
19:	learn: 0.5668948	total: 1

In [54]:
val

,Кол-во отправленных сообщений,Кол-во полученных сообщений,Кол-во Адресатов,Кол-во поздно прочитанных сообщений,Среднее кол-во дней между получением и прочтением,Кол-во отвеченных сообщений,Кол-во символов текста в сообщениях,Соотношение полученных и отправленных
97,349,123,46,3,16,25,29,2.837398
942,538,158,50,15,13,139,1,3.405063
601,747,433,73,47,4,348,30,1.725173
327,128,567,34,12,10,239,53,0.225750
632,932,52,121,108,18,901,24,17.923077
...,...,...,...,...,...,...,...,...
691,1732,688,7,27,8,113,2,2.517442
216,1930,1043,54,57,14,149,12,1.850431
659,38,599,21,0,14,347,15,0.063439
715,59,36,51,41,16,183,21,1.638889


In [63]:
probabilities = [probability[1] * 100 for probability in cb.predict_proba(val)]
probabilities

[6.536057870098048,
 26.383895401188788,
 87.37988285334065,
 75.98818630341016,
 24.176955149075415,
 7.366738308079563,
 13.40952775768641,
 2.731761610639032,
 1.2484783798957746,
 3.6521605364032284,
 9.632052660450556,
 13.428317847054844,
 80.13541225765705,
 3.777308311781389,
 28.269365904713755,
 33.56937002861122,
 74.4114993763907,
 10.602356876401307,
 8.94051976659456,
 27.501617781528054,
 67.99261737008244,
 19.051216862358192,
 4.412438294672874,
 2.3085441911180653,
 34.10456146923419,
 3.083138481961599,
 5.782689412295483,
 16.48075465265636,
 91.86509827198535,
 3.0696396983789915,
 50.51648905874108,
 5.45721360012826,
 9.419172917253936,
 94.58811816284909,
 1.7708845098850636,
 54.717585243393444,
 4.936467700075885,
 17.13903140193368,
 2.392200853573159,
 61.17832427100044,
 7.111818727266922,
 84.86164887061058,
 21.584774911588873,
 48.41456218046043,
 54.37850963228347,
 32.43425008934067,
 67.14504512143442,
 12.927452131130854,
 86.68276143565437,
 10.5993

In [58]:

preds = cb.predict(val)
score = f1_score(val_target, preds)
score

0.36170212765957444